Remove stop words, numbers, punctuations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

import csv
from nltk.corpus import stopwords
import numpy as np
import re
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data_path = '/content/drive/MyDrive/NLP_Workspace/Data/'
with open(data_path + 'balanced_data.csv','r') as file:     ### file path to be changed
  csvreader = csv.reader(file)
  data = np.array(list(csvreader))

with open(data_path + 'emoticons.csv','r', encoding='UTF-8') as file:     ### file path to be changed
  csvreader = csv.reader(file)
  emoticon = np.array(list(csvreader))[1:]

In [ ]:
stoplist = stopwords.words('english')
stoplist.extend(["ever", "one", "do","does","make", "go", "us", "to", "get", 
                 "about", "may", "s", ".", ",", "!", "i", "I", '\"', "?", ";", 
                 "--", "--", "would", "could", "”", "Mr.", "Miss", "Mrs.", 
                 "don’t", "said", "can't", "didn't", "aren't", "I'm", "you're", 
                 "they're", "'s", "(", ")", "...", ":", "'", "+", "n't", "*", "@"
                 ,"[", "]", "'d", "'ve", "..", "=", "+", "-", "_" , "/"
                 ,"'m", "'ll", "^", "%", "......", "...."])
stoplist

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

Emoji positivity extraction algorithm

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 16 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import re
import spacy
from nltk.stem import WordNetLemmatizer
import nltk

with open(data_path + 'emoji_sentiment.csv','r') as file:     ### file path to be changed
  csvreader = csv.reader(file)
  emoji = np.array(list(csvreader))[1:]

nlp = spacy.load("en_core_web_lg", exclude=["tagger", "parser", "senter", "attribute_ruler", "lemmatizer", "ner"])

nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def lemmatized(str):
    return [lemmatizer.lemmatize(token) for token in str.split()]


results = {}


# from name of found emoji, query emoji database to find its underlying positivity
def get_pos(name):
    mx = 0
    mx_pos = 0
    cur = 0
    mem = ''
    override = [0,0,0]
    if name in results:
      # print("%s found in memory" % (name))
      return results.get(name)
    
    cos_name = nlp(" ".join(lemmatized(name.lower())))
    for i in emoji:
        query = lemmatized(i[7].lower())
        querystr = " ".join(query)
        cur = nlp(querystr).similarity(cos_name)
        if name in query and cur > override[1]:
            override = [querystr, cur, int(i[6]) / (int(i[4]) + int(i[5]) + int(i[6]))]
            continue
        if mx < cur:
            mem = querystr
            mx = cur
            mx_pos = int(i[6]) / (int(i[4]) + int(i[5]) + int(i[6]))
    
    if override[1] > 0:
        # print("most similar word to %s is %s with sim of %f, pos of %f." % (name, mem, mx, mx_pos))
        # print("overriden to literal match at %s is %s with sim of %f, pos of %f." % (name, override[0], override[1], override[2]))
        results[name] = override[2]
        return mx_pos
    else:
        # print("most similar word to %s is %s with sim of %f, pos of %f." % (name, mem, mx, mx_pos))
        results[name] = mx_pos
        return mx_pos


def emoji_featurize(data):
    count = 0
    start = 0
    pos = 0
    for m in re.finditer(':[a-zA-z\d_]+:', data):
        count += 1
        start += m.start()
        pos += get_pos(data[m.start()+1:m.end()-1])
    jump = False
    for m in range(len(data)):
        if jump:
            jump = False
            continue
        for i in emoticon:
            e = i[0]
            if m + len(e) > len(data) or len(e) == 0:
                continue
            if data[m:m+len(e)] == e:
                count += 1
                start += m
                pos += get_pos(i[2])
                jump = True
                break

    if count == 0:
      return [0,0,0]

    return [count, start/count/len(data), pos/count]


def emoji_replace(data):
    data = re.sub(':[a-zA-z\d_]+:', '<emoji>', data)
    return data

In [ ]:
def replace_by_pos(data):
    old_data = data
    for m in re.finditer(':[a-zA-z\d_]+:', data):
        pos = get_pos(old_data[m.start()+1:m.end()-1])
        if pos >= 0.33:
            data = re.sub(m.group(0), r'<positive emoji>', data)
        else:
            data = re.sub(m.group(0), r'<negative emoji>', data)

    data = data.split()
    for i in emoticon:
        e = i[0]
        if len(e) < 1:
            continue
        for j in range(len(data)):
            if e == data[j]:
                pos = get_pos(i[2])
                if pos >= 0.33:
                    data[j] = '<positive emoji>'
                else:
                    data[j] = '<negative emoji>'
    data = ' '.join(data)
    
    return data

In [ ]:
# mwe_tokenizer = nltk.tokenize.MWETokenizer([('<', 'positive', 'emoji', '>'), ('<', 'negative', 'emoji', '>')])

In [ ]:
print(len(data))

4


In [ ]:
new_data = []
count = 0
for line in data:
  if count % (200255 // 100) == 0:
    print("%d%s processing finished" % (count // (200255 // 100), '%'))
  count += 1
  post = line[1]
  post = replace_by_pos(post)
  
  # post = re.sub(r"(^| )[0-9]+($| )", r" ", post)  # remove digits
  # tokens = mwe_tokenizer.tokenize(tokens)
  # tokens = [val.replace('_', '') for val in tokens]
  addme = [t.lower() for t in post.split()]
  word_count = len(addme)
  if word_count == 0:
    continue
  new_data.append([line[0], ' '.join(addme), word_count, line[3]])

0% processing finished


<ipython-input-54-1d6aeaced09a>:23: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  cur = nlp(querystr).similarity(cos_name)


1% processing finished
2% processing finished
3% processing finished
4% processing finished
5% processing finished
6% processing finished
7% processing finished
8% processing finished
9% processing finished
10% processing finished
11% processing finished
12% processing finished
13% processing finished
14% processing finished
15% processing finished
16% processing finished
17% processing finished
18% processing finished
19% processing finished
20% processing finished
21% processing finished
22% processing finished
23% processing finished
24% processing finished
25% processing finished
26% processing finished
27% processing finished
28% processing finished
29% processing finished
30% processing finished
31% processing finished
32% processing finished
33% processing finished
34% processing finished
35% processing finished
36% processing finished
37% processing finished
38% processing finished
39% processing finished
40% processing finished
41% processing finished
42% processing finished
4

In [ ]:
for i in new_data[:150]:
  print(i)

['type', 'text', 1, 'label']
['ENFJ', "i went through a break up some months ago. we were together for 4 years and i had planned my life around that relationship. i wasn't the one breaking the relationship as you might imagine and all our...", 38, '0']
['ENFJ', 'enfj puns so many puns.', 5, '0']
['ENFJ', "well i personally don't go that much for attractiveness in general but i can see you have the will to change that and that's good already. may i ask if you want to be with them in a merely sexual...", 40, '0']
['ENFJ', "sorry, not an infp but i'm really into post-rock so i had to post <negative emoji> going to leave this here and sneak out <negative emoji>", 26, '0']
['ENFJ', 'welcome!!!', 1, '0']
['ENFJ', 'wrong thread <negative emoji>', 4, '0']
['ENFJ', "that doesn't sound very enfj i think <positive emoji> i'd never act cold towards a romantic interest. those just get my warm side i think. when i like someone that way all i want to do is to just straight to them...", 42, '0']
['ENF

In [ ]:
header = ['type', 'text', 'word count', 'label']

with open(data_path + "emoji_labeled_data_nonltk_BERT.csv", 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)
    writer.writerows(new_data[1:])